In [1]:
import math
import random
import pickle
import warnings
import datetime
import inflection

import numpy        as np
import pandas       as pd
import scipy.stats  as ss

import seaborn as sns

import matplotlib.pyplot as plt
from tabulate             import tabulate
from IPython.display      import Image
from IPython.core.display import HTML

from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder

from sklearn.ensemble import RandomForestRegressor

from boruta import BorutaPy

import json

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [2]:
import sys
sys.path.append('../')

from webapp.rossmann.Rossmann import Rossmann

In [3]:
pipeline = Rossmann()

In [12]:
sales_test = pd.read_csv('../01-Data/test.csv', low_memory=False)
store_raw = pd.read_csv('../01-Data/store.csv', low_memory=False)

df_teste = sales_test.merge(store_raw, how='left', on='Store')

In [13]:
df_teste = df_teste[df_teste['Open'] != 0]

# Data Cleaning
df1 = pipeline.data_cleaning(df_teste)
# Feature Engineering
df2 = pipeline.feature_engineering(df1)
# Data Preparation
df3 = pipeline.data_preparation(df2)
# Prediction
#dfResponse = pipeline.get_prediction(model, testeRaw, df3)

..\webapp\rossmann\Rossmann.py:102: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df2['week_of_year'] = df2['date'].dt.weekofyear


In [14]:
def get_prediction(model, original_data, test_data):
    # Prediction
    pred = model.predict(test_data)

    # Join pred into original Data
    original_data['prediction'] = np.expm1(pred)

    return original_data

In [15]:
model = pickle.load(open('../webapp/model/1_flc_xgb_tuned.pkl', 'rb'))
# Prediction
dfResponse = get_prediction(model, df_teste, df3)
dfResponse.head()

,id,store,day_of_week,date,open,promo,state_holiday,school_holiday,store_type,assortment,...,year,month,day,week_of_year,year_week,competion_sinse,competion_time_month,promo_since,promo_time_week,prediction
0,1,1,4,2015-09-17,1.0,1,regular day,0,c,basic,...,2015,9,17,38,2015-37,2008-09-01,85,2015-09-14,0,4251.291016
1,2,3,4,2015-09-17,1.0,1,regular day,0,a,basic,...,2015,9,17,38,2015-37,2006-12-01,107,2011-03-28,233,6105.725098
2,3,7,4,2015-09-17,1.0,1,regular day,0,a,extended,...,2015,9,17,38,2015-37,2013-04-01,29,2015-09-14,0,6216.384277
3,4,8,4,2015-09-17,1.0,1,regular day,0,a,basic,...,2015,9,17,38,2015-37,2014-10-01,11,2015-09-14,0,5616.511719
4,5,9,4,2015-09-17,1.0,1,regular day,0,a,extended,...,2015,9,17,38,2015-37,2000-08-01,184,2015-09-14,0,6479.495117


In [16]:
import requests
import json

In [ ]:
### API local

In [19]:
sales_test = pd.read_csv('../01-Data/test.csv', low_memory=False)
store_raw = pd.read_csv('../01-Data/store.csv', low_memory=False)

df_teste = sales_test.merge(store_raw, how='left', on='Store')

storeList = [30, 14, 23]

# Choose Store for Prediction
df_teste = df_teste[df_teste['Store'].isin(storeList)]

# Remove Closed Days
df_teste = df_teste[df_teste['Open'] != 0]
df_teste = df_teste[~df_teste['Open'].isnull()]
df_teste = df_teste.drop('Id', axis=1)

In [20]:
# Convert DataFrame to JSON
data = json.dumps(df_teste.to_dict(orient='records'))

In [21]:
## API Call
url = 'http://localhost:5000/rossmann/predict'
#url = 'https://model-rossmann.herokuapp.com/rossmann/predict'
header = {'Content-Type': 'application/json'}
data = data

r = requests.post(url, data=data, headers=header)
print('Status Code {}'.format(r.status_code))

Status Code 200


In [22]:
df_response = pd.DataFrame(r.json(), columns=r.json()[0].keys())

In [23]:
df_response.head()

,store,day_of_week,date,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,...,year,month,day,week_of_year,year_week,competion_sinse,competion_time_month,promo_since,promo_time_week,prediction
0,14,4,2015-09-17T00:00:00.000Z,1.0,1,regular day,0,a,basic,1300.0,...,2015,9,17,38,2015-37,2014-03-01T00:00:00.000Z,18,2011-09-26T00:00:00.000Z,207,6323.219727
1,23,4,2015-09-17T00:00:00.000Z,1.0,1,regular day,0,d,basic,4060.0,...,2015,9,17,38,2015-37,2005-08-01T00:00:00.000Z,123,2015-09-14T00:00:00.000Z,0,7890.851562
2,30,4,2015-09-17T00:00:00.000Z,1.0,1,regular day,0,a,basic,40.0,...,2015,9,17,38,2015-37,2014-02-01T00:00:00.000Z,19,2014-03-03T00:00:00.000Z,80,4946.782715
3,14,3,2015-09-16T00:00:00.000Z,1.0,1,regular day,0,a,basic,1300.0,...,2015,9,16,38,2015-37,2014-03-01T00:00:00.000Z,18,2011-09-26T00:00:00.000Z,207,6422.070312
4,23,3,2015-09-16T00:00:00.000Z,1.0,1,regular day,0,d,basic,4060.0,...,2015,9,16,38,2015-37,2005-08-01T00:00:00.000Z,123,2015-09-14T00:00:00.000Z,0,7532.273438
